In [1]:
cd ..

/Users/yasas/Documents/Projects/textkit-learn


In [2]:
# import datasets
from transformers import AutoModelForSequenceClassification, AutoTokenizer

from tklearn import datasets
from tklearn.metrics import TextClassificationMetric
from tklearn.nn.trainer import Trainer
from tklearn.nn.evaluator import Evaluator
from tklearn.nn.callbacks import ProgbarLogger
from tklearn.config import config, config_scope

/Users/yasas/Documents/Projects/textkit-learn/venv/lib/python3.11/site-packages/pydantic/_internal/_fields.py:128: UserWarning: Field "model_server_url" has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(
/Users/yasas/Documents/Projects/textkit-learn/venv/lib/python3.11/site-packages/pydantic/_internal/_config.py:317: UserWarning: Valid config keys have changed in V2:
* 'schema_extra' has been renamed to 'json_schema_extra'
  warnings.warn(message, UserWarning)


In [3]:
config['emotion/Trainer'] = {
    'epochs': 2,
}

In [4]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")


def tokenize(texts):
    return tokenizer(texts["text"].tolist(), padding="max_length", truncation=True)

In [5]:
train_dset = (
    datasets.load_dataset('hf', 'dair-ai/emotion', split="train").take(range(100))
    .map(tokenize, batched=True, keep_columns=True)
    .rename_column('label', 'labels')
    .remove_columns(["text"])
)

# train_dset = train_dset[:500].to_pylist()

In [6]:
train_dset

Dataset(/Users/yasas/.tklearn/cache/dataset-mapped-4712c86c82069e1b9a689596132b2037, arrow, 100)

In [7]:
valid_dset = (
    datasets.load_dataset('hf', "dair-ai/emotion", split="validation").take(range(100))
    .map(tokenize, batched=True, keep_columns=True)
    .rename_column('label', 'labels')
    .remove_columns(["text"])
)

In [8]:
len(train_dset), len(valid_dset)

(100, 100)

In [9]:
model = AutoModelForSequenceClassification.from_pretrained(
    "bert-base-uncased",
    num_labels=8,
)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [10]:
callbacks = [
    ProgbarLogger(),
]

with config_scope('emotion'):
    trainer = Trainer(model, callbacks=callbacks)

In [11]:
metric = TextClassificationMetric(num_labels=8)

evaluator = Evaluator(valid_dset, metric=metric, postprocessor='argmax')

In [12]:
trainer.fit(train_dset, evaluator=evaluator)

Train:   0%|          | 0/26 [00:00<?, ?it/s]

Predict:   0%|          | 0/13 [00:00<?, ?it/s]

Predict:   0%|          | 0/13 [00:00<?, ?it/s]

In [13]:
import pandas as pd

In [14]:
test_dset = (
    datasets.load_dataset('hf', "dair-ai/emotion", split="test")
    .map(tokenize, batched=True, keep_columns=True)
    .rename_column('label', 'labels')
    .remove_columns(["text"])
)

groups_df = pd.get_dummies(test_dset.select('labels').to_pandas().iloc[:, 0])

evaluator = Evaluator(test_dset, metric=metric, postprocessor='argmax', groups=groups_df)

In [15]:
evaluator.evaluate(trainer)

Predict:   0%|          | 0/250 [00:00<?, ?it/s]

{'micro': {'f1': 0.36250000000000004, 'precision': 0.3625, 'recall': 0.3625},
 'macro': {'f1': 0.18247451682253937,
  'precision': 0.1675735440270324,
  'recall': 0.20977792136498907},
 'weighted': {'f1': 0.3175804440923456,
  'precision': 0.29409991800834817,
  'recall': 0.3625},
 'accuracy': 0.3625,
 'val_loss': 1.7873933134078979,
 0: {'micro': {'f1': 0.5903614457831325,
   'precision': 0.5903614457831325,
   'recall': 0.5903614457831325},
  'macro': {'f1': 0.2474747474747475,
   'precision': 0.3333333333333333,
   'recall': 0.19678714859437751},
  'weighted': {'f1': 0.7424242424242424,
   'precision': 1.0,
   'recall': 0.5903614457831325},
  'accuracy': 0.5903614457831325},
 1: {'micro': {'f1': 0.4719424460431655,
   'precision': 0.4719424460431655,
   'recall': 0.4719424460431655},
  'macro': {'f1': 0.21375040729879444,
   'precision': 0.3333333333333333,
   'recall': 0.15731414868105517},
  'weighted': {'f1': 0.6412512218963833,
   'precision': 1.0,
   'recall': 0.471942446043165